In [ ]:
!nvidia-smi

Wed Jul 19 21:37:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.8 MB/s eta 0:00:

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset,load_from_disk,load_metric
import matplotlib.pyplot as plt
import pandas as pd
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail" #transformer basede model from hugging face
tokenizer = AutoTokenizer.from_pretrained(model_ckpt) #tokenizing using the same transformer based model.
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device) #loading everything into GPU

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#download and unzip the data
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip #samsum data, again from hugging face

--2023-07-19 21:39:40--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2023-07-19 21:39:40--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.06s   

2023-07-19 21:39:40 (119 MB/s) - ‘summarizer-data.zip’ saved [7903594/

In [ ]:
!unzip summarizer-data.zip

Archive:  summarizer-data.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
   creating: samsum_dataset/
 extracting: samsum_dataset/dataset_dict.json  
   creating: samsum_dataset/test/
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
   creating: samsum_dataset/train/
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
   creating: samsum_dataset/validation/
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [ ]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split Lengths : {split_lengths}")
print(f"Features : {dataset_samsum['train'].column_names}")
print("\n Dialogue")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary : ")
print(dataset_samsum["test"][1]["summary"])


Split Lengths : [14732, 819, 818]
Features : ['id', 'dialogue', 'summary']

 Dialogue
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary : 
Eric and Rob are going to watch a stand-up on youtube.


In [ ]:
def convert_examples_to_features(example_batch) :
  input_encodings = tokenizer(example_batch['dialogue'] , max_length = 512,truncation = True)
  with tokenizer.as_target_tokenizer() :
    target_encoding = tokenizer(example_batch['summary'] , max_length = 128,truncation = True)

  return {
      "input_ids" : input_encodings['input_ids'],
      "attention_mask" : input_encodings['attention_mask'],
      "labels" : input_encodings['input_ids']
  } #we are adding these columns to our training data

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

Training


In [ ]:
from transformers import DataCollatorForSeq2Seq #for taking the data in batches
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer , model = model_pegasus)

In [ ]:
from transformers import TrainingArguments , Trainer
trainer_args = TrainingArguments(
    output_dir = "pegasus-samsum" ,
    num_train_epochs = 1,
    warmup_steps = 500,
    per_device_train_batch_size= 10,
    per_device_eval_batch_size = 10,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy="steps",
    eval_steps = 500,
    save_steps = 1e6,
    gradient_accumulation_steps=128
)
#hyper parameters

In [ ]:
trainer = Trainer(model = model_pegasus ,
                  args = trainer_args ,
                  tokenizer = tokenizer ,
                  data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt['test'],
                  eval_dataset = dataset_samsum_pt['validation'])
#taking training as testing, because training data is too huge and this is a trail ipynb file

In [ ]:
trainer.train()

OutOfMemoryError: ignored